In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
from shapely.geometry import Point,LineString
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval as make_tuple
%matplotlib inline

In [2]:
us_zipcodes = gpd.read_file('tl_2023_us_zcta520/tl_2023_us_zcta520.shp')
us_county = gpd.read_file('tl_2020_us_county/tl_2020_us_county.shp')
us_state = gpd.read_file('tl_2020_us_state/tl_2020_us_state.shp')
zipcode_money_data = pd.read_csv('CleanZipcodeDataNewNew.csv')

In [3]:
us_zipcodes['ZCTA5CE20'] = us_zipcodes['ZCTA5CE20'].astype(int)
zipcode_money_data['ZCTA5CE20'] = zipcode_money_data['ZCTA5CE20'].astype(int)
zipcode_data = us_zipcodes.merge(zipcode_money_data,how = 'right' ,on = 'ZCTA5CE20')
zipcode_data.shape[0]

206

In [4]:
zipcode_data['Centroid'] = np.where(zipcode_data['geometry'] != None , 
                                    zipcode_data['geometry'].to_crs('+proj=cea').centroid.to_crs(zipcode_data['geometry'].crs), None)


In [5]:
missing_zipcode_data = pd.read_csv('missing_zipcode_data.csv')


In [6]:
missing_zipcode_data['point'] = ''
for i in range(38):
    missing_zipcode_data['point'][i] = list(missing_zipcode_data['centerpoint'][i].split(", "))
    for j in range(2):
        missing_zipcode_data['point'][i][j] = float(missing_zipcode_data['point'][i][j])
        
    missing_zipcode_data['point'][i] = tuple(missing_zipcode_data['point'][i])
    

/var/folders/7t/y5fl09j137v1_49j8q0zc3900000gn/T/ipykernel_14316/3074340002.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_zipcode_data['point'][i] = list(missing_zipcode_data['centerpoint'][i].split(", "))
/var/folders/7t/y5fl09j137v1_49j8q0zc3900000gn/T/ipykernel_14316/3074340002.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_zipcode_data['point'][i] = tuple(missing_zipcode_data['point'][i])


In [7]:
lat = []
long = []
for cord in missing_zipcode_data['point']:
    lat.append(cord[0])
    long.append(cord[1])
    
missing_zipcode_data['Latitude'] = lat
missing_zipcode_data['Longitude'] = long

missing_zipcode_data_gpd = gpd.GeoDataFrame(
    missing_zipcode_data, geometry=gpd.points_from_xy(missing_zipcode_data.Longitude, missing_zipcode_data.Latitude), crs="EPSG:4269"
)


final_missing = missing_zipcode_data_gpd.drop(['centerpoint','point','Latitude','Longitude'],axis = 1)
final_missing['Centroid'] = final_missing['geometry']

In [8]:
final_data = zipcode_data.merge(final_missing, on = 'ZCTA5CE20', how = 'left', suffixes = ('','_y'))
final_data['Centroid'] = np.where(final_data['geometry'] == None, final_data['Centroid_y'], final_data['Centroid'])

In [9]:
data = final_data.drop([12,29,70])
# we deleted zipcodes 28950, 70149, 78200
# these are indexes 12, 29, 70
data = data.drop(['geometry_y', 'Centroid_y'],axis = 1)


In [10]:
data[['2019-2020','2020-2021','2021-2022','2022-2023','2023-2024','Total']] = data[['2019-2020','2020-2021','2021-2022','2022-2023','2023-2024','Total']].apply(pd.to_numeric)

In [13]:
#data['Size'] = data['Total']/10000

#data.to_csv('Final_Data.csv')

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands